In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
np.random.seed(42)

In [6]:
x = np.random.uniform(0, 1, 50) # 50 data points using uniform distribution
x[:5]

array([0.96958463, 0.77513282, 0.93949894, 0.89482735, 0.59789998])

In [8]:
# Remember: In reality we don't have this function
def f(x):
    return 3*x + 2

In [11]:
t = f(x)
t[:5]

array([4.90875388, 4.32539847, 4.81849682, 4.68448205, 3.79369994])

In [16]:
noise = np.random.normal(0, 0.3, size=x.shape[0])
# Reality target values have noises

t = t + noise
t[:5]

array([4.61549355, 4.59156937, 4.76899518, 5.34949011, 3.51620111])